In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import affine_transform
import time

# Reuse the RobustInverseCompositional and PyramidalInverseCompositional layers from previous code

class TFDSEvaluator:
    def __init__(self, img_size=(256, 256)):
        self.img_size = img_size
        self.models = {
            'Single-scale': RobustInverseCompositional(
                transform_type=TransformType.AFFINE,
                TOL=1e-5,
                robust_type=RobustErrorFunctionType.HUBER,
                lambda_=0.1,
                max_iter=50
            ),
            'Pyramidal': PyramidalInverseCompositional(
                nscales=3,
                nu=0.5,
                transform_type=TransformType.AFFINE,
                TOL=1e-5,
                robust_type=RobustErrorFunctionType.HUBER,
                lambda_=0.1,
                max_iter=50
            )
        }
        
        # Load dataset
        self.dataset = self.load_dataset()

    def load_dataset(self):
        """Load horses_or_humans dataset from TFDS"""
        ds = tfds.load('horses_or_humans', split='test', shuffle_files=True)
        
        def preprocess(example):
            img = tf.image.convert_image_dtype(example['image'], tf.float32)
            img = tf.image.resize(img, self.img_size)
            return img.numpy()  # Convert to numpy for scipy transformations
        
        return [preprocess(x) for x in ds]

    def generate_affine_params(self):
        """Generate random affine transformation parameters"""
        return {
            'scale': np.random.uniform(0.7, 1.3, 2),
            'rotation': np.random.uniform(-np.pi/6, np.pi/6),
            'shear': np.random.uniform(-0.3, 0.3),
            'translation': np.random.uniform(-50, 50, 2)
        }

    def create_affine_matrix(self, params):
        """Create affine transformation matrix from parameters"""
        scale_matrix = np.array([[params['scale'][0], 0, 0],
                                [0, params['scale'][1], 0]])
        
        rotation_matrix = np.array([[np.cos(params['rotation']), -np.sin(params['rotation']), 0],
                                   [np.sin(params['rotation']), np.cos(params['rotation']), 0]])
        
        shear_matrix = np.array([[1, params['shear'], 0],
                                [0, 1, 0]])
        
        translation_matrix = np.array([[1, 0, params['translation'][0]],
                                      [0, 1, params['translation'][1]]])
        
        # Combine transformations: scale -> rotate -> shear -> translate
        matrix = translation_matrix @ shear_matrix @ rotation_matrix @ scale_matrix
        return matrix[:2, :]

    def apply_affine_transform(self, img, matrix):
        """Apply affine transform using scipy"""
        transformed = np.zeros_like(img)
        for c in range(3):
            transformed[..., c] = affine_transform(
                img[..., c],
                matrix=matrix[:, :2],
                offset=matrix[:, 2],
                order=3,
                mode='constant',
                cval=0.0
            )
        return transformed

    def evaluate(self, num_samples=50):
        results = {name: {'mse': [], 'mae': [], 'epe': [], 'time': []} 
                  for name in self.models}
        
        for i in range(num_samples):
            # Select random image
            img1 = self.dataset[np.random.randint(len(self.dataset))]
            
            # Generate random transformation
            params = self.generate_affine_params()
            matrix = self.create_affine_matrix(params)
            
            # Apply transformation
            img2 = self.apply_affine_transform(img1, matrix)
            
            # Convert to tensors
            I1 = tf.expand_dims(tf.convert_to_tensor(img1, tf.float32), 0)
            I2 = tf.expand_dims(tf.convert_to_tensor(img2, tf.float32), 0)
            
            # Ground truth parameters (flattened affine matrix)
            true_params = matrix.flatten()[:6]

            for name, model in self.models.items():
                start_time = time.time()
                
                # Estimate transformation
                p_pred, error, DI, Iw = model([I1, I2])
                params_pred = p_pred.numpy()[0]
                
                # Calculate parameter errors
                mse = np.mean((params_pred - true_params)**2)
                mae = np.mean(np.abs(params_pred - true_params))
                
                # Calculate endpoint error (EPE)
                pred_flow = self.params_to_flow(params_pred)
                true_flow = self.params_to_flow(true_params)
                epe = np.mean(np.sqrt(np.sum((pred_flow - true_flow)**2, axis=-1)))
                
                # Store results
                results[name]['mse'].append(mse)
                results[name]['mae'].append(mae)
                results[name]['epe'].append(epe)
                results[name]['time'].append(time.time() - start_time)
                
        return results

    def params_to_flow(self, params):
        """Convert affine parameters to flow field"""
        h, w = self.img_size
        matrix = params.reshape(2, 3)
        
        x = np.linspace(0, w-1, w)
        y = np.linspace(0, h-1, h)
        xx, yy = np.meshgrid(x, y)
        ones = np.ones_like(xx)
        coords = np.stack([xx, yy, ones], axis=0)
        
        transformed = np.tensordot(matrix, coords, axes=([1], [0]))
        flow = np.moveaxis(transformed, 0, -1) - np.stack([xx, yy], axis=-1)
        return flow

    def plot_results(self, results):
        metrics = ['mse', 'mae', 'epe', 'time']
        fig, axs = plt.subplots(2, 2, figsize=(15, 10))
        
        for i, metric in enumerate(metrics):
            ax = axs[i//2, i%2]
            for name in self.models:
                ax.plot(results[name][metric], label=name)
            ax.set_title(f'{metric.upper()} Comparison')
            ax.set_xlabel('Sample Index')
            ax.set_ylabel(metric.upper())
            ax.legend()
            ax.grid(True)
            
        plt.tight_layout()
        plt.show()




In [ ]:
# Run evaluation
evaluator = TFDSEvaluator()
results = evaluator.evaluate(num_samples=50)
evaluator.plot_results(results)


In [ ]:
# Print summary statistics
print("Average Metrics:")
for name in evaluator.models:
    print(f"\n{name}:")
    for metric in ['mse', 'mae', 'epe', 'time']:
        avg = np.mean(results[name][metric])
        print(f"  {metric.upper()}: {avg:.4f}")